In [1]:
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import json
import pickle
import joblib
import os
from skimage.measure import ransac
from src.triangulate import triangulate_joints, simple_processor, ransac_processor
from src.utils import plot_keypoints, reprojection_error

# Define run parameters

In [2]:
# Parameters
params = joblib.load("./hands_maximo/calib/cam_data.pkl")
frame = "00000"
root_dir = "./hands_maximo"
handedness = "right"
conf_thresh_start = 0.75
min_tri_kp = 10

# Create projection matrices

In [3]:
proj_mats = {}
for cam_name, cam_param in params[str(int(frame))].items():
    K =  np.identity(3)
    K[0,0] = cam_param['K'][0]
    K[1,1] = cam_param['K'][1]
    K[0,2] = cam_param['K'][2]
    K[1,2] = cam_param['K'][3]
    proj_mats[cam_name] = K @ cam_param['extrinsics_opencv']

# Get 2D keypoints

In [4]:
keypoints = {}
for cam_name in proj_mats.keys():
    with open(f"{root_dir}/keypoints_2d/{cam_name}/{frame}.json", "r") as f:
        data = json.load(f)
        keypoints[cam_name] = np.array(data['people'][0][f'hand_{handedness}_keypoints_2d']).reshape(-1, 3)

num_joints = keypoints[cam_name].shape[0]

# Triangulate
## Naive Triangulation

In [5]:
keypoints3d, residuals = triangulate_joints(keypoints, proj_mats, processor=simple_processor)
print(residuals.mean())

6.0111525230986205


## RANSAC Triangulation

In [6]:
keypoints3d, residuals = triangulate_joints(keypoints, proj_mats, processor=ransac_processor, residual_threshold=10, min_samples=5)
print(residuals.mean())

5.429630254000197
